# Assignment 2

## Start with data review and cleanup. Last step in this process is to create the train, test and validate datasets that are saved to the data/processed folder

In [81]:
# Import the Pandas and Numpy libraries
# Configure auto-reload for the session

import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
#Import the raw data from /data/raw folder

raw_datafile = "../data/raw/beer_reviews.csv"
df_raw = pd.read_csv(raw_datafile)

In [86]:
# Inspect the data

df_raw.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [3]:
df_raw.shape

# 13 Columns
# 1,586,614 rows

(1586614, 13)

In [4]:
df_raw.describe()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
count,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.518829e+06,1.586614e+06
mean,3.130099e+03,1.224089e+09,3.815581e+00,3.735636e+00,3.841642e+00,3.743701e+00,3.792860e+00,7.042387e+00,2.171279e+04
std,5.578104e+03,7.654427e+07,7.206219e-01,6.976167e-01,6.160928e-01,6.822184e-01,7.319696e-01,2.322526e+00,2.181834e+04
min,1.000000e+00,8.406720e+08,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,3.000000e+00
25%,1.430000e+02,1.173224e+09,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00,1.717000e+03
50%,4.290000e+02,1.239203e+09,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00,1.390600e+04
75%,2.372000e+03,1.288568e+09,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00,3.944100e+04
max,2.800300e+04,1.326285e+09,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01,7.731700e+04


# Key Observations

Brewey ID and Beer ID are identifiers and could be removed
beer_name, brewery_name, beer_style, review_profilename are category style columns
Review_taste and Review_palate  have values between 1 and 5
Since the target value (beer_style) is a category, need to further analyze the values in this column

In [83]:
# Create a new dataframe with only the fields that will be expected to participate in the model

fields_used_in_model = ["brewery_name", "review_aroma", "review_appearance", "review_palate", "review_taste", "beer_style"]
df_source = df_raw[fields_used_in_model]
df_source.head()

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_style
0,Vecchio Birraio,2.0,2.5,1.5,1.5,Hefeweizen
1,Vecchio Birraio,2.5,3.0,3.0,3.0,English Strong Ale
2,Vecchio Birraio,2.5,3.0,3.0,3.0,Foreign / Export Stout
3,Vecchio Birraio,3.0,3.5,2.5,3.0,German Pilsener
4,Caldera Brewing Company,4.5,4.0,4.0,4.5,American Double / Imperial IPA


In [84]:
# Define the variables that identify the category and numeric columns so we can apply scaling
cat_cols = ["brewery_name"]
num_cols = ["review_aroma", "review_appearance", "review_palate", "review_taste"]
target_col = ["beer_style"]

In [10]:
df_source.describe()

,review_aroma,review_appearance,review_palate,review_taste
count,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06
mean,3.735636e+00,3.841642e+00,3.743701e+00,3.792860e+00
std,6.976167e-01,6.160928e-01,6.822184e-01,7.319696e-01
min,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00
50%,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
75%,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00
max,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00


In [22]:
df_source.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1586592 entries, 0 to 1586613
Data columns (total 6 columns):
brewery_name         1586592 non-null object
review_aroma         1586592 non-null float64
review_appearance    1586592 non-null float64
review_palate        1586592 non-null float64
review_taste         1586592 non-null float64
beer_style           1586592 non-null object
dtypes: float64(4), object(2)
memory usage: 84.7+ MB


In [85]:
#Drop rows with 0 for review_appearance values as there's not enough data to include these in the model
df_source.drop(df_source.loc[df_source['review_appearance'] == 0.0].index, axis=0, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [86]:
#remove rows that contain missing values
df_source.dropna(subset=cat_cols, inplace=True)
df_source.reindex()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_style
0,Vecchio Birraio,2.0,2.5,1.5,1.5,Hefeweizen
1,Vecchio Birraio,2.5,3.0,3.0,3.0,English Strong Ale
2,Vecchio Birraio,2.5,3.0,3.0,3.0,Foreign / Export Stout
3,Vecchio Birraio,3.0,3.5,2.5,3.0,German Pilsener
4,Caldera Brewing Company,4.5,4.0,4.0,4.5,American Double / Imperial IPA
...,...,...,...,...,...,...
1586609,The Defiant Brewing Company,4.0,3.5,4.0,4.0,Pumpkin Ale
1586610,The Defiant Brewing Company,5.0,2.5,2.0,4.0,Pumpkin Ale
1586611,The Defiant Brewing Company,3.5,3.0,3.5,4.0,Pumpkin Ale
1586612,The Defiant Brewing Company,4.5,4.5,4.5,4.5,Pumpkin Ale


In [87]:
#Save the uencoded data before encoding. This will be needed to test the pipeline later in the process
df_source.to_csv("../data/interim/source_unencoded.csv")

In [23]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

df_source[num_cols] = sc.fit_transform(df_source[num_cols])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [24]:
#inspect the data to ensure that the numeric feilds are now scaled
df_source.head()

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_style
0,Vecchio Birraio,-2.487987,-2.177891,-3.288890,-3.132518,Hefeweizen
1,Vecchio Birraio,-1.771254,-1.366253,-1.090151,-1.083220,English Strong Ale
2,Vecchio Birraio,-1.771254,-1.366253,-1.090151,-1.083220,Foreign / Export Stout
3,Vecchio Birraio,-1.054520,-0.554615,-1.823064,-1.083220,German Pilsener
4,Caldera Brewing Company,1.095680,0.257023,0.375676,0.966078,American Double / Imperial IPA


In [27]:
#Encode the Brewery Name field with Ordinal Encoder and save the encoder for future use in the MODELS folder
from sklearn.preprocessing import OrdinalEncoder
import joblib

oe = OrdinalEncoder()

bname_encoder = oe.fit(df_source[cat_cols])
joblib.dump(bname_encoder, "../models/bname.joblib")

['../models/bname.joblib']

In [28]:
#Test the model with valid and invalid values. Invalid values returns errors - need to figure out how to handle the error and return -1 instead
float(bname_encoder.transform([["Caldera Brewing Company"]]))

1480.0

In [29]:
#Transform the bewery_name column to now included the encoded values based on the model that we created earlier
df_source[cat_cols] = bname_encoder.transform(df_source[cat_cols])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [30]:
#inspect the df_source to confirm the encoding of brewery name is successful
df_source.head()

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_style
0,5438.0,-2.487987,-2.177891,-3.288890,-3.132518,Hefeweizen
1,5438.0,-1.771254,-1.366253,-1.090151,-1.083220,English Strong Ale
2,5438.0,-1.771254,-1.366253,-1.090151,-1.083220,Foreign / Export Stout
3,5438.0,-1.054520,-0.554615,-1.823064,-1.083220,German Pilsener
4,1480.0,1.095680,0.257023,0.375676,0.966078,American Double / Imperial IPA


In [31]:
#Now use the LabelEncoder to encode beer_style data in the same manner
from sklearn.preprocessing import LabelEncoder
import joblib

le = LabelEncoder()

beerstyle_encoder = le.fit(df_source[target_col])
joblib.dump(beerstyle_encoder, "../models/beerstyle.joblib")

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['../models/beerstyle.joblib']

In [33]:
#Now encode all beer style values in the main dataset
df_source[target_col] = beerstyle_encoder.transform(df_source[target_col])

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
#Save the final data set before we split this off into train, test and validate sets
df_source.to_csv("../data/interim/source.csv")

In [35]:
#Final inspection of the data before we break it up into train, test and validate sets
df_source.describe()

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_style
count,1.586592e+06,1.586592e+06,1.586592e+06,1.586592e+06,1.586592e+06,1.586592e+06
mean,2.905174e+03,1.156508e-16,1.056191e-16,-1.633728e-17,-1.360724e-16,4.214240e+01
std,1.729726e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.302087e+01
min,0.000000e+00,-3.921453e+00,-4.612805e+00,-4.021804e+00,-3.815618e+00,0.000000e+00
25%,1.325000e+03,-3.377870e-01,-5.546151e-01,-3.572372e-01,-4.001210e-01,1.200000e+01
50%,2.880000e+03,3.789463e-01,2.570229e-01,3.756761e-01,2.829783e-01,3.100000e+01
75%,4.581000e+03,3.789463e-01,2.570229e-01,3.756761e-01,9.660777e-01,7.400000e+01
max,5.741000e+03,1.812413e+00,1.880299e+00,1.841503e+00,1.649177e+00,1.030000e+02


In [36]:
from src.data.sets import split_sets_random, save_sets

X_train, Y_train, X_valid, Y_valid, X_test, Y_test = split_sets_random(df_source, target_col="beer_style")
save_sets(X_train, Y_train, X_valid, Y_valid, X_test, Y_test)

## This is the final step in data cleanup and preparation. Next step is to start training an algorithm and test accuracy

In [37]:
#First step is to train a null model
from src.models.null import NullModel

base_model = NullModel(target_type="classification")
Y_base = base_model.fit_predict(Y_train)

In [38]:
from src.models.performance import print_class_perf

print_class_perf(y_preds=Y_base, y_actuals=Y_train, set_name='Training', average='weighted')

Accuracy Training: 0.07409286583175237
F1 Training: 0.010222119412199842


## Null model provides a baseline accuracy of 7.4%.

In [74]:
#Next step is to train a linear regression model. Once the model is trained, predict against the test and validate datasets and check the accuracy score

from sklearn.linear_model import LinearRegression
from src.models.performance import print_reg_perf

reg_model = LinearRegression()

reg_model.fit(X_train, Y_train)

Y_test_preds = reg_model.predict(X_test)
Y_valid_preds = reg_model.predict(X_valid)


print_reg_perf(y_preds=Y_test_preds, y_actuals=Y_test, set_name='Testing')
print_reg_perf(y_preds=Y_valid_preds, y_actuals=Y_valid, set_name='Validation')

RMSE Testing: 32.98422040754792
MAE Testing: 29.425972028427083
RMSE Validation: 32.965301630817024
MAE Validation: 29.41851810424962


In [67]:
print(reg_model.intercept_)
print(reg_model.coef_)
feature_cols = ["brewery_name","review_aroma","review_appearance","review_palate","review_taste"]
list(zip(feature_cols, reg_model.coef_))

44.95960402483058
[-0.00097089 -0.11893497 -0.30478868  0.03838517 -0.08451253]


[('brewery_name', -0.0009708866636696669),
 ('review_aroma', -0.11893497076815845),
 ('review_appearance', -0.30478868216062244),
 ('review_palate', 0.03838517351342658),
 ('review_taste', -0.08451253052055477)]

In [61]:
#Save the base linear regression model
from joblib import dump
dump(reg_model, "../models/linear_regress1.joblib")

['../models/linear_regress1.joblib']

In [3]:
#Next stop is to use Logistic Regression to see if we can get better outcome
from sklearn.linear_model import LogisticRegression
from src.models.performance import print_reg_perf

logistic_reg = LogisticRegression()

logistic_reg.fit(X_train, Y_train)

Y_test_preds_logistic = logistic_reg.predict(X_test)
Y_valid_preds_logistic = logistic_reg.predict(X_valid)

print_reg_perf(y_preds=Y_test_preds_logistic, y_actuals=Y_test, set_name='Testing')
print_reg_perf(y_preds=Y_valid_preds_logistic, y_actuals=Y_valid, set_name='Validation')

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


RMSE Testing: 44.694053560549456
MAE Testing: 32.382046514559434
RMSE Validation: 44.73441097070581
MAE Validation: 32.43413273666961


# Scores for the basic Logistic Regression model are better than Linear Regression. Continue with hyper parameter tuning of this model to see if we can better outcome

In [10]:
from sklearn.linear_model import LogisticRegression

logistic_reg_ovr = LogisticRegression(multi_class='ovr')

logistic_reg_ovr.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
from src.models.performance import print_reg_perf

Y_test_preds_ovr = logistic_reg_ovr.predict(X_test)
Y_valid_preds_ovr = logistic_reg_ovr.predict(X_valid)

print_reg_perf(y_preds=Y_test_preds_ovr, y_actuals=Y_test, set_name='Testing')
print_reg_perf(y_preds=Y_valid_preds_ovr, y_actuals=Y_valid, set_name='Validation')

RMSE Testing: 45.43884792547178
MAE Testing: 32.99478759611748
RMSE Validation: 45.457511783388135
MAE Validation: 33.01863733770327


In [12]:
from sklearn.metrics import accuracy_score

print("Accuracy Score - Testing: ", accuracy_score(Y_test, Y_test_preds_ovr))
print("Accuracy Score - Validation: ", accuracy_score(Y_valid, Y_valid_preds_ovr))

Accuracy Score - Testing:  0.08982730366822135
Accuracy Score - Validation:  0.09023698474725829


In [6]:
from sklearn.linear_model import LogisticRegression

logistic_reg_multi = LogisticRegression(multi_class='multinomial', solver='saga', random_state=8, penalty='none', max_iter=200)

logistic_reg_multi.fit(X_train, Y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='multinomial', n_jobs=None, penalty='none',
                   random_state=8, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [7]:
from src.models.performance import print_reg_perf

Y_test_preds_multi = logistic_reg_multi.predict(X_test)
Y_valid_preds_multi = logistic_reg_multi.predict(X_valid)

print_reg_perf(y_preds=Y_test_preds_multi, y_actuals=Y_test, set_name='Testing')
print_reg_perf(y_preds=Y_valid_preds_multi, y_actuals=Y_valid, set_name='Validation')

RMSE Testing: 46.20337944516929
MAE Testing: 33.68314950207992
RMSE Validation: 46.24289285849065
MAE Validation: 33.737655993949325


In [8]:
from sklearn.metrics import accuracy_score

print("Accuracy Score - Testing: ", accuracy_score(Y_test, Y_test_preds_multi))
print("Accuracy Score - Validation: ", accuracy_score(Y_valid, Y_valid_preds_multi))


Accuracy Score - Testing:  0.08420206731375268
Accuracy Score - Validation:  0.08497415857809151


## Above scores are a very small improvement from the default Logistic Regression model. Next we will try XGBoost

In [33]:
#Train an XGBoost model with default hyper parameters
import xgboost as xgb
xgb_model1 = xgb.XGBClassifier()

xgb_model1.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [34]:
Y_test_preds_xgb1 = xgb_model1.predict(X_test)
Y_val_preds_xgb1 = xgb_model1.predict(X_valid)

In [91]:
from src.models.performance import print_class_perf

print_class_perf(y_preds=Y_test_preds_xgb1, y_actuals=Y_test, set_name='Testing', average='weighted')
print_class_perf(y_preds=Y_val_preds_xgb1, y_actuals=Y_valid, set_name='Validation', average='weighted')

Accuracy Training: 0.038992184545569145
F1 Training: 0.03799543039970671
Accuracy Validation: 0.039666582629522246
F1 Validation: 0.03883663030002738


In [84]:
dump(xgb_model1, "../models/xgb_model1.joblib")

['../models/xgb_model1.joblib']

In [98]:
#Next experiment is to figure out the best value of various XGBoost hyper parameters

#Define the search space for various parameters
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin

xgb_space = {
    'max_depth' : hp.choice('max_depth', range(5, 20, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.05)
}

In [99]:
def xgb_objective(space):
    from sklearn.model_selection import cross_val_score
    
    xgbmodel = xgb.XGBClassifier(
        max_depth = int(space['max_depth']),
        learning_rate = space['learning_rate']
    )
    
    acc = cross_val_score(xgbmodel, X_train, Y_train, cv=3, scoring="accuracy").mean()

    return{'loss': 1-acc, 'status': STATUS_OK }

In [ ]:
best = fmin(
    fn=xgb_objective,   
    space=xgb_space,       
    algo=tpe.suggest,       
    max_evals=5
)

In [1]:
from src.data.sets import load_sets

X_train, Y_train, X_valid, Y_valid, X_test, Y_test = load_sets()


## Try Naive Bayes algorithm next

In [11]:
from sklearn.naive_bayes import GaussianNB

nbclassifier = GaussianNB()
nbclassifier.fit(X_train, Y_train)
Y_test_predict = nbclassifier.predict(X_test)

acc_score = nbclassifier.score(X_test, Y_test)
print("Number of mislabeled points out of a total %d points : %d" %(X_test.shape[0], (Y_test != Y_test_predict).sum()))

Number of mislabeled points out of a total 317320 points : 288878


In [6]:
print(acc_score)

0.08963191730744989


In [8]:
acc_score = nbclassifier.score(X_valid, Y_valid)
print(acc_score)

0.08945859069708811


## Try RandomForest Classifier

In [39]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=104, max_samples=500)

rfc.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=3000,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=104,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [40]:
# Run predictions for Test and Validation sets

Y_test_predict1 = rfc.predict(X_test)
Y_valid_predict1 = rfc.predict(X_valid)


In [41]:
#Calcuate and print the accuracy of the RFC model

from src.models.performance import print_class_perf

print_class_perf(y_preds=Y_test_predict1, y_actuals=Y_test, set_name='Training', average='weighted')
print_class_perf(y_preds=Y_valid_predict1, y_actuals=Y_valid, set_name='Validation', average='weighted')

Accuracy Training: 0.15948934668267578
F1 Training: 0.13376825546806112
Accuracy Validation: 0.16071524239014998
F1 Validation: 0.13515928043514533


### RFC offers a accuracy score better than baseline model at around 16% across the test and validation sets.

In [42]:
# Save the RFC model for future reference

from joblib import dump
dump(rfc, "../models/rfc.joblib")

['../models/rfc.joblib']

In [43]:
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': 3000,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 104,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [55]:
#Train a second RFC model with hyperparameter tuning that allows a more dense tree to be created

from sklearn.ensemble import RandomForestClassifier

rfc1 = RandomForestClassifier(n_estimators=104, max_samples=500, min_samples_leaf=10)

rfc1.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=3000,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=104,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [56]:
Y_test_predict2 = rfc1.predict(X_test)
Y_valid_predict2 = rfc1.predict(X_valid)


In [57]:
from src.models.performance import print_class_perf

print_class_perf(y_preds=Y_test_predict2, y_actuals=Y_test, set_name='Training', average='weighted')
print_class_perf(y_preds=Y_valid_predict2, y_actuals=Y_valid, set_name='Validation', average='weighted')

Accuracy Training: 0.12143930870827149
F1 Training: 0.07209497205522031
Accuracy Validation: 0.12263999319296985
F1 Validation: 0.07320997553556982


### Since the accurace of this model drops compared to previous model, we'll use the first RFC model for prediction

### The cells below are purely for testing purposes. I used these to confirm the various commands for the Main.py module

In [75]:
from app.feature_encode import BreweryNameEncodedVal

In [78]:
bn = BreweryNameEncodedVal()
brew_name = pd.DataFrame(["Caldera Brewing Company"])
brew_code = bn.fit(brew_name)

In [79]:
brew_code

1480.0

In [51]:
s_test = "[{\"brewery_name\": \"ABC1\", \"rev_aroma\": 3.5, \"rev_appearance\": 2.5, \"rev_palate\": 4.0, \"rev_taste\": 5.0 }, {\"brewery_name\": \"ABC2\", \"rev_aroma\": 3.5, \"rev_appearance\": 2.5, \"rev_palate\": 4.0, \"rev_taste\": 5.0}, {\"brewery_name\": \"ABC3\", \"rev_aroma\": 3.5, \"rev_appearance\": 2.5, \"rev_palate\": 4.0, \"rev_taste\": 5.0}]"

In [52]:
import pandas as pd
df_test = pd.read_json(s_test)

In [53]:
df_test

,brewery_name,rev_aroma,rev_appearance,rev_palate,rev_taste
0,ABC1,3.5,2.5,4,5
1,ABC2,3.5,2.5,4,5
2,ABC3,3.5,2.5,4,5


In [54]:
df_test.loc[0]

brewery_name      ABC1
rev_aroma          3.5
rev_appearance     2.5
rev_palate           4
rev_taste            5
Name: 0, dtype: object

In [55]:
df_test.shape[0]

3

In [56]:
df_test.loc[1]["brewery_name"]

'ABC2'

In [57]:
strResult = ""
max_index = df_test.shape[0]
print(max_index)
for i in range (0, max_index):
    print(df_test.loc[i]["brewery_name"])
    strResult += df_test.loc[i]["brewery_name"]
    if i < max_index-1:
        strResult += ", "

print (strResult)

3
ABC1
ABC2
ABC3
ABC1, ABC2, ABC3


In [58]:
def format_features(brewery_name: str, rev_aroma: float, rev_taste: float, rev_appearance: float, rev_palate: float):
    return {
        "brewery_name": [brewery_name],
        "review_aroma": [rev_aroma],
        "review_taste": [rev_taste],
        "review_appearance": [rev_appearance],
        "review_palate": [rev_palate]
    }

In [59]:
from feature_encode import BeerStyleCode
from joblib import load
import pandas as pd

rfc_pipeline = load("../models/pipeline_model_rfc.joblib")
beer_style_decode = BeerStyleCode()

beer_list_in_json = "[{\"brewery_name\": \"Caldera Brewing Company\", \"rev_aroma\": 3.5, \"rev_appearance\": 2.5, \"rev_palate\": 4.0, \"rev_taste\": 5.0 }, {\"brewery_name\": \"Amstel Brouwerij B. V.\", \"rev_aroma\": 3.5, \"rev_appearance\": 2.5, \"rev_palate\": 4.0, \"rev_taste\": 5.0}, {\"brewery_name\": \"Kirner Bier\", \"rev_aroma\": 3.5, \"rev_appearance\": 2.5, \"rev_palate\": 4.0, \"rev_taste\": 5.0}]"
    
df_beer = pd.read_json(beer_list_in_json) # this function returns a dataframe with the features entered
max_index = df_beer.shape[0] #Get the number of records in the dataframe
beer_style_result = ""
for i in range (0, max_index):  # For each record, create a prediction and append it to the beer_style_result string
    brewery_name = df_beer.loc[i]["brewery_name"]
    review_aroma = df_beer.loc[i]["rev_aroma"]
    review_taste = df_beer.loc[i]["rev_taste"]
    review_palate = df_beer.loc[i]["rev_palate"]
    review_appearance = df_beer.loc[i]["rev_appearance"]
    features = format_features(brewery_name, review_aroma, review_taste, review_appearance, review_palate)
    obs = pd.DataFrame(features)
    pred_beer_style = beer_style_decode.inverse_transform(rfc_pipeline.predict(obs))
    beer_style_result += pred_beer_style["beer_style"]
    if i < max_index-1:
        beer_style_result += ", "
        
print(beer_style_result)

0    American IPA, American Double / Imperial Stout...
Name: beer_style, dtype: object


/opt/conda/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


In [96]:
beer_list_in_json = "[{%22brewery_name%22:%20%22Caldera%20Brewing%20Company%22,%20%22rev_aroma%22:%203.5,%20%22rev_appearance%22:%202.5,%20%22rev_palate%22:%204.0,%20%22rev_taste%22:%205.0%20},%20{%22brewery_name%22:%20%22Amstel%20Brouwerij%20B.%20V.%22,%20%22rev_aroma%22:%203.5,%20%22rev_appearance%22:%202.5,%20%22rev_palate%22:%204.0,%20%22rev_taste%22:%205.0},%20{%22brewery_name%22:%20%22Kirner%20Bier%22,%20%22rev_aroma%22:%203.5,%20%22rev_appearance%22:%202.5,%20%22rev_palate%22:%204.0,%20%22rev_taste%22:%205.0}]"

In [97]:
from feature_encode import BeerStyleCode
from joblib import load
import pandas as pd

beer_style_list = []

rfc_pipeline = load("../models/pipeline_model_rfc.joblib")
beer_style_decode = BeerStyleCode()

df_beer = pd.read_json(beer_list_in_json) # this function returns a dataframe with the features entered
max_index = df_beer.shape[0] #Get the number of records in the dataframe
beer_style_result = ""
for i in range (0, max_index):  # For each record, create a prediction and append it to the beer_style_result string
    brewery_name = df_beer.loc[i]["brewery_name"]
    review_aroma = df_beer.loc[i]["rev_aroma"]
    review_taste = df_beer.loc[i]["rev_taste"]
    review_palate = df_beer.loc[i]["rev_palate"]
    review_appearance = df_beer.loc[i]["rev_appearance"]
    features = format_features(brewery_name, review_aroma, review_taste, review_appearance, review_palate)
    obs = pd.DataFrame(features)
    pred_beer_style = beer_style_decode.inverse_transform(rfc_pipeline.predict(obs))
    beer_style_list.append(pred_beer_style)
    #beer_style_result += pred_beer_style["beer_style"]
    if i < max_index-1:
        beer_style_result += ", "
        
print(pred_beer_style)

ValueError: Expected object or value

In [75]:
df_1 = pd.read_json(beer_list_in_json)

ValueError: Expected object or value

In [74]:
type(beer_list_in_json)

str

In [76]:
import json

In [79]:
json.dump(beer_list_in_json, str)

AttributeError: type object 'str' has no attribute 'write'

In [81]:
df_test["rev_aroma"].to_json()

'{"0":3.5,"1":3.5,"2":3.5}'

In [85]:
dummy_list = []
for i in range (0, 5):
    dummy_list.append("A")
    print(dummy_list[i])

beers_df = pd.DataFrame(dummy_list, columns=["beer_style"]) #initialize an empty dataframe
print(beers_df.head())
print(beers_df["beer_style"].to_string())


A
A
A
A
A
  beer_style
0          A
1          A
2          A
3          A
4          A
0    A
1    A
2    A
3    A
4    A


In [90]:
str(beers_df["beer_style"])

'0    A\n1    A\n2    A\n3    A\n4    A\nName: beer_style, dtype: object'

In [95]:
from feature_encode import BeerStyleCode
import pandas as pd

beer_style_decode = BeerStyleCode()
beer_style = beer_style_decode.inverse_transform(1)




ValueError: bad input shape ()